In [304]:
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

In [305]:
links = None

with open('links.json', 'r') as f:
    links = json.loads(f.read())

links

{'Hindi-Persian': 'https://en.m.wiktionary.org/wiki/Category:Hindi_terms_borrowed_from_Persian',
 'English-French': 'https://en.m.wiktionary.org/wiki/Category:English_terms_borrowed_from_French',
 'Finnish-Swedish': 'https://en.m.wiktionary.org/wiki/Category:Finnish_terms_borrowed_from_Swedish'}

In [306]:
base_link = "https://en.m.wiktionary.org"

In [307]:
def get_words_ipas(dest_lang, src_lang, link, page_num):
    borrowed_words = []

    try:
        # get list of words
        page = requests.get(link)
        soup = BeautifulSoup(page.content, 'html.parser')
        res = soup.find_all("div", class_="mw-category-group")

        for i, r in enumerate(res):
            if i > 0:
                list_words = r.find_all('a')
                src_word = None

                for borrowed_word in list_words:

                    # for each loaned word get the wiki link and extract ipas for it.
                    ipa_page = requests.get(base_link + borrowed_word['href'])
                    ipa_soup = BeautifulSoup(ipa_page.content, 'html.parser')


                    # get the src word and wiki link to extract ipas for the src word
                    src_word_html_final = None
                    for header in ipa_soup.find_all("span", class_="mw-headline"):
                        if re.match("Etymology [1-9]",header.text):
                            if dest_lang == header.parent.parent.find_previous_siblings()[0].find_all("span", class_="mw-headline")[0].text:
                                if len(header.parent.find_next_siblings()) > 0:
                                    if len(header.parent.find_next_siblings()[0].find_all("span", class_="etyl")) > 0:
                                        etym = header.parent.find_next_siblings()[0].find_all("span", class_="etyl")[0]
                                        for tag in etym.find(lambda tag: tag.name == 'a') :
                                            if src_lang.lower() not in tag.text.lower():
                                                src_word_html_final = etym.find_next("i")
                                                meaning = header.parent.find_next_siblings('ol')[0].text.split('\n')[0]
                                                meaning = re.sub(r'\(.+?\)',r'',meaning)
                                                meaning = re.sub(r'\[.+?\]',r'',meaning)
                                                meaning = meaning.replace('  ', ' ')
                                                meaning = re.split('[.;:]',meaning)[0].strip()
                                                src_word = src_word_html_final.text if src_word_html_final != None else ''
                                                src_word = re.split('[,;:]',src_word)[0].strip()
                                                if len(borrowed_words) == 0 or \
                                                    borrowed_word.text != borrowed_words[-1][0]:
                                                    print(borrowed_word.text,src_word,tag.text.lower(),meaning)
                                                    borrowed_words.append([borrowed_word.text,src_word,tag.text.lower(),meaning]) 

        print(dest_lang, src_lang, page_num, "done", len(borrowed_words), "\n")
        # return borrowed_words      
        # find whether there is a next page containing more loan words if there is scrape that as well. 
        next_link = soup.find(
            lambda tag: tag.name == 'a' and 'next page' in tag.text.lower()
        )

        # return borrowed_words
        if next_link != None:
            more_words = get_words_ipas(dest_lang, src_lang, base_link + next_link['href'], page_num+1)
            borrowed_words = borrowed_words + more_words

    except Exception as e:
        # errors can be due to timeouts/connections refused due to rate limiting. can be fixed via proxies/vpns/timeouts
        print(pair, page_num, "got error", e, len(borrowed_words))
        return borrowed_words
        
    return borrowed_words

In [308]:
for pair in links:
    [dest, src] = pair.split('-')
    words = get_words_ipas(dest, src, links[pair], 1)
    df = pd.DataFrame(words, columns=['loan_word', 'original_word', 'other_etymology', 'other_meaning'])
    df.to_csv("results/{}_false_friends.csv".format(pair), index=False)
    print(pair, "done\n")

अगर अगरु sanskrit agarwood
आचार आचार sanskrit conduct, behavior
-आना -𑀆𑀯𑁂𑀤𑀺 sauraseni prakrit forms transitive verbs
आराम आराम sanskrit sensual pleasure, delight, bliss
आलू 𑀆𑀮𑀼𑀕 sauraseni prakrit potato
-ई -𑀇𑀅 sauraseni prakrit relating to, forms adjectives from nouns
-ईन -ईन sanskrit relating to
कमान command english command
काफ़ी coffee english Alternative form of कॉफ़ी
कार कार sanskrit action, doing
Hindi Persian 1 done 10 

-कारी -कारी sanskrit -er
कुंद 𑀓𑀼𑀁𑀤 sauraseni prakrit a kind of flower, Jasminum multiflorum
कूर क्रूर sanskrit cruel
कोह 𑀓𑁄𑀥 sauraseni prakrit anger, fury
ग़रारा غَرْغَرَة arabic gargle, gargling
गिलास glass english glass, tumbler
चीज़ cheese english cheese
Hindi Persian 2 done 7 

ज़ार czar english czar
जान ज्ञान sanskrit knowledge
जाली जाली sanskrit lattice
तार तार sanskrit the सप्तक of highest pitch
तीर तीर sanskrit shore
तोप स्तूप sanskrit heap, pile, cluster
दफ़ा دُفْعَة arabic occasion, time
दम दम sanskrit self-restraint, taming, self-control
दराज़ drawers 

English French 16 done 4 

lisse līssian old english To relieve, mitigate, assuage
lurch lurcāre latin To swallow or eat greedily
Lyon lyon middle english A surname, from nicknames​
macaroon macarone italian A coarse, rude, low fellow
Maire Máire irish A female given name from Irish
maki 巻き japanese makizushi, a form of sushi that is rolled
English French 17 done 6 

Maura Maura catalan A surname​
Metis Μῆτις ancient greek A Titan who was the personification of wisdom, the first spouse of Zeus, and the mother of Athena
metis μῆτις ancient greek Practical intelligence
metre metrum old english The rhythm or measure in verse and musical composition
Milan Milano italian The capital city of Lombardy, Italy
Milot Milot albanian A town in northwestern Albania, located just south of the left bank of the river Mat near the Zogu Bridge below the western slope of the Mali e Sukës
mine min middle english That which belongs to me
Misson mos old english A village and civil parish in Bassetlaw distri